<h1 align='center'> Performance Metrics for Regression Problems </h1>

In this note book, I am going to illsutrate how we can evaluate the ML models deployed for regression task using different kind of evaluation metrics.

For illsutareting purpose I have collected the data from kaggle. I will be doing all the analysis over them only. I have attach the link for each dataset, you can also download the same.


For the implemetations of these metrics, I am using following tools and frameworks:
- Python - as a primary language
- Pandas - as an analytical engine for processing the data
- numpy - for computation using numpy arrays
- matplotlib - for plotting the figures
- sklearn - for implememting the metrics
- seaborn - for graph plotting 

Note that I will be implementing all the metrices from scratch.



Let's write some generic code which will be used throughout this notebook.


## Contents

1. [Imports](#imports)
1. [Data Loader](#data-loader)
1. [Basic Exploratory Data Analysis (EDA)](#basic-exploratory-data-analysis-eda)
1. [Modeling](#modeling)
    1. [Train/Test Splits](#traintest-split)
    1. [Model Selection](#model-selection)
    1. [Model Training](#model-training)
1. [Model Evaluation](#model-evaluation)
    1. [Mean Squared Error](#mean-squared-error-mse)
    1. [Root Mean Squared Error](#root-mean-squared-errorrmse)
    1. [Mean Absolute Error](#mean-absolute-error-mae)  
    1. [R-squared scores](#r-squared)
    1. [Pearson correlation](#Pearson-correlation)
    1. [Spearman rank correlation](#Spearman-rank-correlation)
    1. [Adjusted R-squared](#adjusted-r-squared)

## Imports


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import sklearn

## Data Loader


In [2]:
#function for loading a csv file from specified path

def data_loader(path):
    
    #use pandas to load the data from path
    df = pd.read_csv(path)
    
    return df


<h2 align='center'>Boston Housing Price Prediction</h2>

**Problem Statement :**

Given the information about locality, crimes and other relevant information about locality of Boston City(USA) predict a range of the  price of the houses.

**About Data :** 

The Boston Housing Dataset is a derived from information collected by the U.S. Census Service concerning housing in the area of Boston MA. The Data is from kaggle and same can be downloaded from <a href="https://www.kaggle.com/code/kmalit/bank-customer-churn-prediction">here</a>.

**Columns Info:**

 The following describes the dataset columns:

- **CRIM -** per capita crime rate by town
- **ZN -** proportion of residential land zoned for lots over `25,000` sq.ft.
- **INDUS -** proportion of non-retail business acres per town.
- **CHAS -** Charles River dummy variable (`1` if tract bounds river; `0` otherwise)
- **NOX -** nitric oxides concentration (parts per `10` million)
- **RM -** average number of rooms per dwelling
- **AGE -** proportion of owner-occupied units built prior to `1940`
- **DIS -** weighted distances to five Boston employment centres
- **RAD -** index of accessibility to radial highways
- **TAX -** full-value property-tax rate per `$10,000`
- **PTRATIO -** pupil-teacher ratio by town
- **B -** `1000` `(Bk - 0.63)^2` where `Bk` is the proportion of blacks by town
- **LSTAT -** % lower status of the population
- **MEDV -** Median value of owner-occupied homes in `$1000`'s

**Variable of Interest Seggregation**

For the problem statement and data info we can deduce following:
- **Target Variable -** `Median value of owner-occupied homes (MEDV)`
- **Independent Variables -** `Rest of the columns (CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, RAD, TAX, PTRATIO, B, LSTAT)`


**Algorithms to be used :**

Since this is a regression problem I will be using `Linear Regression`, `Polynomial Regression`, `Random Forest Regressor` and `XGBoost Regressor` which are based on Regression Algorithms for modeling pourpose.

**Evaluations metrics :**

Since this is a regression problem, I will be using following metrics for evaluating output of the predictive models.

- Mean Squared Error(MSE)
- Root Mean Squared Error(RMSE)
- Mean Absolute Error(MAE)
- Coefficient of Determinations(R^2)
- Adjusted R^2
- Huber Loss
- Mean Percentage Error(MAE)


`Always look at your data`
## Load the data

I have downloaded the data from above link and have stored the same in my local file system. 

In [3]:
#df contains all the data
path= "/Users/ajitkumarsingh/Desktop/Data-Science-Interview-Questions/performance-metrics/data/HousingData.csv"
df = data_loader(path)

# show first row
df.head(1)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0


### Basic Exploratory Data Analysis (EDA)

In [4]:
#columns name in churn df
print(f"columns name : {', '.join(df.columns.tolist())}\n")

#total number of columns
print(f"total number of columns : {len(df.columns)}\n")

#total count of the data
print(f"Number of rows in data : {len(df)}\n")



columns name : CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, RAD, TAX, PTRATIO, B, LSTAT, MEDV

total number of columns : 14

Number of rows in data : 506



We have 14 attributes and 10000 rows in the churn data.

In [5]:
#number of nulls per attributes
df.isnull().sum()

CRIM       20
ZN         20
INDUS      20
CHAS       20
NOX         0
RM          0
AGE        20
DIS         0
RAD         0
TAX         0
PTRATIO     0
B           0
LSTAT      20
MEDV        0
dtype: int64

We have some null values in the data. The ML models are not directly deal with null values, we need to either drop the rows where null values is present in atleast one of the columns or impute the null values with appropriate values.
There are several ways to impute the missing values, Here I am imputing the missing values with the average value of the their entire column. 

I am using `df.fillna()` to accomplish it.

In [6]:
#impute the null values with mean

df = df.fillna(df.mean())

In [7]:
#number of nulls per attributes
df.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

Now, we don't have null values in any of the columns.

In [8]:
# number of unique values
df.nunique()

CRIM       485
ZN          27
INDUS       77
CHAS         3
NOX         81
RM         446
AGE        349
DIS        412
RAD          9
TAX         66
PTRATIO     46
B          357
LSTAT      439
MEDV       229
dtype: int64

We have some columns like `CRIM` and `LSTAT` with high cardinality. High cardinalty means we have many unique values in that columns and high cardinality might lead to increase in model complexity. It is advisable to reduce the cardinality using `binning` or `discretization` if the we are not so concerned about literal values of that attribute.



In [9]:
# data types
df.dtypes

CRIM       float64
ZN         float64
INDUS      float64
CHAS       float64
NOX        float64
RM         float64
AGE        float64
DIS        float64
RAD          int64
TAX          int64
PTRATIO    float64
B          float64
LSTAT      float64
MEDV       float64
dtype: object

All columns seems to have `numeric` data type which is a good thing because we can directly feed these values to our models and we don't have to worry about encoding and all.



### Feature Selection

This is the crucial part of a ML modeling. Our main main objective is to predict the price of the houses based on some attributes and it is important to recognise the important features which are key driving factor of the pricing. There are many mathematical and analytical ways to do that, like we can build a correlation matrix and we can find the correlation coefficient to see which columns are more correlated to target variable and also we can figure out correlated columns and can drop any of them to reduce the overall compleity of the model. Another way to use your domain knowledge and analyze each attributes and see if it impacts the target in any way if not drop the feature.


For time being I am asssuming all the columns present in `df` are equaly important and are key driving factor in the target value prediction.

In [10]:
redundant_features = []

#drop redundant features(in this case it is empty)
df = df.drop(redundant_features, axis=1)

#show 1 row
df.head(1)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0


### Features Extraction

Feature extraction is a process in machine learning and data analysis that involves transforming raw data or input features into a set of representative features that are more meaningful and informative for a particular task or problem. It is an important step in the feature engineering process, which involves selecting, transforming, and creating new features from raw data to improve the performance and interpretability of machine learning models.

This this problem we noted that we do have some columns which have like high cardinality and we can reduce cardinality using `binning` process.

In `binning` method, we convert `numeric` columns to `categorical` columns to reduce overall cardinality of an attribute.

Let's say, if the number of distinct values in a column is greater than the `50%` of the total count it means that column has high cardinality

In [11]:
# number of unique values
df.nunique()

CRIM       485
ZN          27
INDUS       77
CHAS         3
NOX         81
RM         446
AGE        349
DIS        412
RAD          9
TAX         66
PTRATIO     46
B          357
LSTAT      439
MEDV       229
dtype: int64

In [12]:
#columns with high cardinality
total_count = df.shape[0]
unique_counts = df.nunique()

#filter for high cardinality
high_cardinality_columns = unique_counts[unique_counts>0.5*total_count].index.tolist()

print("Columns with high cardinalities are {}".format(', '.join(high_cardinality_columns)))

Columns with high cardinalities are CRIM, RM, AGE, DIS, B, LSTAT


Discreatize the categorical columns one by one.

In [13]:
#dicreatization based on quatiles
#classify crimes(CRIM) into HIGH, Medium and Low
df['CRIM_BINNED'] = pd.qcut(df['CRIM'],q=3,labels=['Low', 'Moderate', "High"])

#bin the age coliumn into Young, Adult and Old
df['AGE_BINNED'] = pd.qcut(df['AGE'],q=3,labels=['Young', 'Adult', "Old"])

#bin the average number of room per dwelling(RM) into Low, Moderate and High
df['RM_BINNED'] = pd.qcut(df['RM'], q=3, labels=['Low', 'Moderate', "High"])

#bin distance column (DIS) into Very Near, Near, Far and Very Far
df['DIS_BINNED'] = pd.qcut(df['DIS'], q=4, labels=['Very Near', 'Near', "Far", "Very Far"])

#bin Black population proportion
df['B_BINNED'] = pd.qcut(df['B'], q=2, labels=['Low', 'High'])

#bin % of lower status of the popultion (LSTAT)
df['LSTAT_BINNED'] = pd.qcut(df['LSTAT'], q=3, labels=['Low', 'Medium','High'])




**Note** that there might be many permutation and combinations here like what is optimal number of categories a column should be binned. For time being I have used common intution for transforming the above high cardinality columns.

Also note that while binning we are replacing the numeric values to some non-numeric data type mainly strings here. We need to encode them before we feed the data to our ML models.

There are several techniques for encoding the values into numeric once. Here I am using `One Hot Encoding` method to label the categorical values. In pandas we have `pd.get_dummies(df, columns=column_list)` to get dummies numeric values for non-numeric values. 

This method in pandas append new columns and the number of these new columns are only dependent on the number of `unique` values in the original columns. For example suppose we have a column name `Gender` and it has two distinct values `Male` and `Female`, this methods will create two new columns namely `Gender_Male` and `Gender_Female` and values of `Gender_Male` will be `0` where `Gender` column is `Female` and `1` if the value is `Male`. 

In [14]:
#remove the high catdinality columns now
df_binned = df.drop(high_cardinality_columns, axis=1)

Let's see how many unique values we have per attributes after binning high cardinality columns.

In [15]:
#number of unique values
df_binned.nunique()

ZN               27
INDUS            77
CHAS              3
NOX              81
RAD               9
TAX              66
PTRATIO          46
MEDV            229
CRIM_BINNED       3
AGE_BINNED        3
RM_BINNED         3
DIS_BINNED        4
B_BINNED          2
LSTAT_BINNED      3
dtype: int64

In [16]:
#Encode the categorica columns now using One Hot Encoding
suffix = '_BINNED'
binned_columns = list(filter(lambda x : x[-len(suffix):]==suffix, df.columns))

#use pd.get_dummies to encode the above columns
df_encoded = pd.get_dummies(df, columns=binned_columns)

#now let's see if we have new columns 

In [17]:
#now see the columns name

df_encoded.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV', 'CRIM_BINNED_Low',
       'CRIM_BINNED_Moderate', 'CRIM_BINNED_High', 'AGE_BINNED_Young',
       'AGE_BINNED_Adult', 'AGE_BINNED_Old', 'RM_BINNED_Low',
       'RM_BINNED_Moderate', 'RM_BINNED_High', 'DIS_BINNED_Very Near',
       'DIS_BINNED_Near', 'DIS_BINNED_Far', 'DIS_BINNED_Very Far',
       'B_BINNED_Low', 'B_BINNED_High', 'LSTAT_BINNED_Low',
       'LSTAT_BINNED_Medium', 'LSTAT_BINNED_High'],
      dtype='object')

Clearly `pd.get_dummies` method has appended some new columns in the data.

Just make sure all the columns are of `numeric` data type.

In [18]:
#data types of final df
df_encoded.dtypes

CRIM                    float64
ZN                      float64
INDUS                   float64
CHAS                    float64
NOX                     float64
RM                      float64
AGE                     float64
DIS                     float64
RAD                       int64
TAX                       int64
PTRATIO                 float64
B                       float64
LSTAT                   float64
MEDV                    float64
CRIM_BINNED_Low           uint8
CRIM_BINNED_Moderate      uint8
CRIM_BINNED_High          uint8
AGE_BINNED_Young          uint8
AGE_BINNED_Adult          uint8
AGE_BINNED_Old            uint8
RM_BINNED_Low             uint8
RM_BINNED_Moderate        uint8
RM_BINNED_High            uint8
DIS_BINNED_Very Near      uint8
DIS_BINNED_Near           uint8
DIS_BINNED_Far            uint8
DIS_BINNED_Very Far       uint8
B_BINNED_Low              uint8
B_BINNED_High             uint8
LSTAT_BINNED_Low          uint8
LSTAT_BINNED_Medium       uint8
LSTAT_BI

We don't have any non numeric data type columns.

## Modeling

### Train/Test Split

For evaluating a model, we need some data to test it on, once training part is done. We usually split the data into two parts i.e train and test. 
On train data we update the models parameters and on test data we see how the trained model is performing. 

For spliting the data, I am using `train_test_split` function of module `sklearn.model_selection`. 

Also we need to segregate the `label` from the rest of the data.


In [19]:
from sklearn.model_selection import train_test_split

#split the data into test and train in 85:15 ratio and drop the label from the rest of the data

train_data, test_data, train_labels, test_labels = train_test_split(
        df_encoded.drop(['MEDV'], axis=1), 
        df_encoded['MEDV'], 
        test_size=0.15, 
        random_state=42
    )

#count after split
print(f"Total count : {len(df_encoded)}\n")
print(f"Train Data count : {len(train_data)}\n")
print(f"Test Data count : {len(test_data)}\n")
print(f"Train Labels count : {len(train_labels)}\n")
print(f"Test Labels count : {len(test_labels)}\n")

# number of rows and labels should match
assert len(train_data)==len(train_labels)
assert len(test_data)==len(test_labels)


Total count : 506

Train Data count : 430

Test Data count : 76

Train Labels count : 430

Test Labels count : 76



We have stored train and test data in `train_data`, `test_data` and train and test labels in `train_labels`, `test_labels`. I will be using them during training and testing time accordingly.

### Model Selection

We have numerous models out there to solve same type of poblems. But the interesting part is like we don't know which model will be best fitting our dataset and perform well on the test dataset. We need to make choice here and to do that we need to evaluate these models one by one using some performance metrics.

I will be training following models and evaluating their performance on test data

- Linear Regressor
- Random Forest Regressor
- Gradient Boosting Regressor
- Neighrest Neighbour Regressor

Here, I will be using `sklearn` module of `scikit-learn` library to implement the above mentioned models.

In [20]:
#import the above models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [26]:
#create a list that contains one instances of each model type

model_instances = {
          'linear_regressor':LinearRegression(),
          'random_forest_regressor': RandomForestRegressor(n_estimators=4, max_depth=4, max_features=4), 
          'knn_regressor':KNeighborsRegressor(), 
          'gradient_boosting_regressor':GradientBoostingRegressor(n_estimators=4, max_depth=4)
          }


### Model Training

In the above step we have created instances of each model and stored in `model_instances` dict type variable. Now we need to train these instances by feeding the train distribution. 

Once the training is done save the trained instances in the same dict `model_instances`

In [27]:
#train the models on training set and save the trained model for evaluation

for model_name, model in model_instances.items():

    #fit the model with train_data and train_labels
    model.fit(train_data,train_labels)
    
    #save the trained model
    model_instances[model_name] = model
    

### Metrics For Model Evaluation

Now we have trained our models and they are ready to make predictions over test dataset. To make sure the model is predicting meaningfull values not random output we need some metrics to evaluate the output. This is where performance metrics come into picture. 

Let's implement some of the most widely used evaluation metrics for regresion problems from scratch.

#### Performance Metrics Implementation

##### Mean Squared Error (MSE)

It estimated as average squared differences of predicted values and the actual values. For an ideal model `mse` would be equal to `0`. Lower `mse` depicts more accurate predictions. It is expressed in squared units of the target variable

Mathematically it is expressed as :

$$
    MSE = \frac{1}{n} \sum_{i=1}^{n} (y_{\text{pred}, i} - y_{\text{true}, i})^2
$$

Note that the above expression is `differential` and hence can we calculate `gradient` for optimizing purposes.



In [28]:
"""
    Calculate Mean Squared Error (MSE) between true and predicted values.

    Parameters:
        -- y_true (numpy array or list): True values
        -- y_pred (numpy array or list): Predicted values

    Returns:
        -- mse (float): Mean Squared Error
    """

def mean_squared_error(y_true, y_pred):

    #convert then into umpy array if not already
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    #squared the differences
    squared_diff = (y_true - y_pred)**2
    #find the avg of squared_diff
    mse = np.mean(squared_diff)
    return mse


**Bounds of mean-squared error**

[0, $\infty$), with 0 the best.

**Value encoded by mean-squared error**

This measure seeks to summarize the errors made by a regression classifier. The smaller it is, the closer the model's predictions are to the truth. In this sense, it is intuitively like a counterpart to [accuracy](#Accuracy) for classifiers.

**Weaknesses of mean-squared error**

These values are highly dependent on scale of the output variables, making them very hard to interpret in isolation. One really needs a clear baseline, and scale-independent ways of comparing scores are also needed.

**Related to mean-squared error**

Scikit-learn implements a variety of closely related measures: [mean absolute error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html#sklearn.metrics.mean_absolute_error), [mean squared logarithmic error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_log_error.html#sklearn.metrics.mean_squared_log_error), and [median absolute error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.median_absolute_error.html#sklearn.metrics.median_absolute_error). I'd say that one should choose among these metrics based on how the output values are scaled and distributed. For instance:

* The median absolute error will be less sensitive to outliers than the others.
* Mean squared logarithmic error might be more appropriate where the outputs are not strictly speaking linearly increasing.

**Cautions**

- **Sensitive to outliers:** MSE is sensitive to outliers because if a predicted value (`y_pred_i`) is an outlier, the squared difference with the corresponding true value (`y_true_i`) will be larger, potentially resulting in an inflated MSE.

- **Imbalanced errors:** MSE treats all errors equally, regardless of their magnitude or direction. This means that overestimation and underestimation errors are weighted equally, even though they may have different implications or costs in practice.

- **Not robust to non-Gaussian errors:** MSE assumes that the errors follow a Gaussian (normal) distribution, which may not always be the case in real-world scenarios. If the errors are not normally distributed, MSE may not accurately reflect the model's performance.

- **Lack of sensitivity to small errors:** MSE may not capture small errors well, as it squares the differences between predicted and true values. This can lead to a model with good MSE but poor performance in capturing small errors, which may be important in certain applications.



##### Root Mean Squared Error(RMSE)

This is root of mean squared error we talked about. The good thing about this metric is like the unit is in sync with target variable and hence can be used for comparing multiple models at once. Since `RMSE` is expressed in the same units as the dependent variable, making it easy to interpret in the context of the original data

The expression of `RMSE` is differentiable just like we have for `MSE`.

In [29]:
"""
    Calculate Root Mean Squared Error (RMSE) between true and predicted values.

    Parameters:
        -- y_true (numpy array or list): True values
        -- y_pred (numpy array or list): Predicted values

    Returns:
        -- mse (float): Mean Squared Error
    """

def root_mean_squared_error(y_true, y_pred):

    #convert then into umpy array if not already
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    #squared the differences
    squared_diff = (y_true - y_pred)**2
    #find the avg of squared_diff
    mse = np.mean(squared_diff)
    return np.sqrt(mse)


**Cautions**

- Not robust to outliers
- Always positive so not usefull when negative errors make sense.
- Bias towards large values
- Can be affected by sample size

##### Mean Absolute Error (MAE)

It is the avearge of absolute difference of predicted and actual values. It is more robust to outliers unlike MSE and RMSE.

Mathematically, It can be expressed as:

$$
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_{\text{true}_i} - y_{\text{pred}_i}|
$$

Note that it is not differential at zero because of the mod `|.|` function.



In [30]:
"""
    Calculate Mean Absolute Error (MAE) between true and predicted values.

    Parameters:
        -- y_true (numpy array or list): True values
        -- y_pred (numpy array or list): Predicted values

    Returns:
        -- mae (float): Mean Absolute Error
    """

def mean_absolute_error(y_true, y_pred):

    #convert then into umpy array if not already
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    #squared the differences
    absolute_diff = np.absolute(y_true - y_pred)
    #find the avg of squared_diff
    mae = np.mean(absolute_diff)
    return mae

##### Coeffiecient Of Determination (R^2)

It is a statistical measure used to evaluate the `goodness of fit` of a regression model. It represents the proportion of the variance in the dependent variable that is explained by the independent variables in the model. It is more usefull when we have linear relationship between dependent and independent variables.

R-squared ranges from `0` to `1`, with `1` indicating a perfect fit and `0` indicating no fit at all.

Mathematially, It can be expressed as:

$$
\text{R-squared} = 1 - \frac{SSR}{SST} = 1.0 - \frac{
      \sum_{i}^{N} (y_{i} - \hat{y_{i}})^{2}     
    }{
       \sum_{i}^{N} (y_{i} - \mu)^{2}
    }
$$

Where :

- *SSR(Sum of Squared Residuals)* represents the sum of the squared differences between the predicted values and the actual values of the dependent variable.

- *SST(Sum of Squared Total)* represents the sum of the squared differences between the actual values of the dependent variable and the mean of the dependent variable.

* $R^{2}$ is [closely related to the squared Pearson correlation coefficient](https://en.wikipedia.org/wiki/Coefficient_of_determination#As_squared_correlation_coefficient).


In [31]:
"""
    Calculate R-squared value given predicted value and actual label

    Parameters:
        -- y_true (numpy array or list): True values
        -- y_pred (numpy array or list): Predicted values

    Returns:
        -- r^2 (float): Coefficient of determination(R^2)
    """

def r2_score(y_true, y_pred):

    #convert then into umpy array if not already
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    #sum of squared residuals
    SSR = np.sum((y_true - y_pred)**2)
    #sum of squared total
    y_avg = np.mean(y_true)
    SST = np.sum((y_avg-y_true)**2)
    return 1 - (float(SSR)/float(SST))

**Note :** `R-squared` value can be negative due to outliers i.e when `MSE(model) > MSE(Baseline)`

**Bounds of R-squared scores**

[0, 1], with 0 the worst and 1 the best.

**Value encoded by R-squared scores**

The numerator in the R$^{2}$ calculation is the sum of errors: 

$$
\textbf{r2}(y, \widehat{y}) =
    1.0 - \frac{
      \sum_{i}^{N} (y_{i} - \hat{y_{i}})^{2}     
    }{
       \sum_{i}^{N} (y_{i} - \mu)^{2}
    }
$$ 

In the context of regular linear regression, the model's objective is to minimize the total sum of squares, which is the denominator in the calculation. Thus, $R^{2}$ is based in the ratio between what the model achieved and what its objective was, which is a measure of the goodness of fit of the model.

**Weaknesses of R-squared scores**

For comparative purposes, it's nice that $R^{2}$ is scaled between [0, 1]; as noted above, this lack of scaling makes mean squared error hard to interpret. But this also represents a trade-off: $R^{2}$ doesn't tell us about the magnitude of the errors.

Limitations:

- **Lack of information on prediction accuracy:** R-squared does not provide information about the accuracy of the model in making predictions. A high R-squared value does not necessarily guarantee accurate predictions, as the model may still have residual errors or may be overfitting the data.

- **Sensitivity to outliers:** R-squared can be affected by outliers in the data. Outliers can disproportionately influence the sum of squared residuals (SSR) component of R-squared, leading to an inflated or deflated R-squared value. 

- **Inability to determine causality:** R-squared does not provide information about causality. Even if a regression model has a high R-squared value, it does not necessarily mean that the independent variables are causing the observed variation in the dependent variable. There may be other confounding variables or omitted variables that are driving the relationship.

- **Limited to linear relationships:** R-squared is a measure of the goodness of fit of a linear regression model, which assumes a linear relationship between the dependent and independent variables. If the relationship is non-linear, R-squared may not accurately reflect the goodness of fit.

##### Adjusted R-squared 

`R-squared` suffers from problem that the scores improve on increasing number of predictors even if the additional predictors do not significantly improve the model's ability to explain the variation in the dependent variable. This can result in overfitting, where the model appears to fit the data well but may not generalize well to new data.

To overcome the above problem associated with `R-squared`, `Adjusted R-squared` adjusts for the number of predictors in the model, penalizing models with more predictors if the additional predictors do not contribute significantly to the model's ability to explain the variation in the dependent variable.

Mathematically It can be expressed as:

$$
\text{Adjusted R-squared} = 1 - \left( \frac{{(1 - R^2) \cdot (n - 1)}}{{n - k - 1}} \right)

$$

Where:

- R-squared (Goodness of Fit): It represents the proportion of variance in the dependent variable explained by the regression model.

- n: Total number of observations in the dataset.

- k: number of predictors(independent variables) in the regression model.



In [32]:
"""
    Calculate Adjusted R-squared value given predicted and actual label and number of predictors

    Parameters:
        -- y_true (numpy array or list): True values
        -- y_pred (numpy array or list): Predicted values
        -- k (integer): Number of predictors

    Returns:
        -- adjusted r^2 (float): Coefficient of determination(R^2)
    """

def adjusted_r2_score(y_true, y_pred, k):

    #convert then into umpy array if not already
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    #number of observations
    n = len(y_true)
    #sum of squared residuals
    SSR = np.sum((y_true - y_pred)**2)
    #sum of squared total
    y_avg = np.mean(y_true)
    SST = np.sum((y_avg-y_true)**2)
    
    #r2_score
    r2_score = 1 - (float(SSR)/float(SST))
    #adjusted r2 square
    r2_score_adj = 1 - float((1-r2_score**2)*(n-1))/float(n - k - 1)
    
    return r2_score_adj

**Note** `Adjusted R_squared` value is sensitive to sample size. It means the value may decrease as we increase the sample size even if the model is performing better.

##### 

#### Pearson correlation

The [Pearson correlation coefficient $\rho$](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) between two vectors $y$ and $\widehat{y}$ of dimension $N$ is:

$$
\textbf{pearsonr}(y, \widehat{y}) = 
\frac{
  \sum_{i}^{N} (y_{i} - \mu_{y}) \cdot (\widehat{y}_{i} - \mu_{\widehat{y}})
}{
  \sum_{i}^{N} (y_{i} - \mu_{y})^{2} \cdot (\widehat{y}_{i} - \mu_{\widehat{y}})^{2}
}
$$
where $\mu_{y}$ is the mean of $y$ and $\mu_{\widehat{y}}$ is the mean of $\widehat{y}$.

This is implemented as `scipy.stats.pearsonr`, which returns the coefficient and a p-value.

##### Bounds of Pearson correlations

$[-1, 1]$, where $-1$ is a complete negative linear correlation, $+1$ is a complete positive linear correlation, and $0$ is no linear correlation at all.

##### Weaknesses of Pearson correlation

Pearson correlations are highly sensitive to the magnitude of the differences between the gold and predicted values. As a result, they are also very sensitive to outliers.

##### Related to Pearson correlation

* For comparing gold values $y$ and predicted values $\widehat{y}$, Pearson correlation is equivalent to a linear regression using $\widehat{y}$ and a bias term to predict $y$. [See this great blog post for details.](https://lindeloev.github.io/tests-as-linear/)

* [As noted above](#Related-to-R-squared-scores), there is also a close relationship to R-squared values.

#### Spearman rank correlation

The Spearman rank correlation coefficient between between two vectors $y$ and $\widehat{y}$ of dimension $N$ is the Pearson coefficient with all of the data mapped to their ranks.


It is implemented as `scipy.stats.spearmanr`, which returns the coefficient and a p-value.


In [1]:
corr_df = pd.DataFrame({
    'y1': np.random.uniform(-10, 10, size=1000),
    'y2': np.random.uniform(-10, 10, size=1000)})

NameError: name 'pd' is not defined

In [2]:
scipy.stats.spearmanr(corr_df['y1'], corr_df['y2'])

NameError: name 'scipy' is not defined

In [3]:
scipy.stats.pearsonr(corr_df['y1'].rank(), corr_df['y2'].rank())

NameError: name 'scipy' is not defined

##### Bounds of Spearman rank correlations

$[-1, 1]$, where $-1$ is a complete negative linear correlation, $+1$ is a complete positive linear correlation, and $0$ is no linear correlation at all.

##### Weaknesses of Spearman rank correlation

Unlike Pearson, Spearman is not sensitive to the magnitude of the differences. In fact, it's invariant under all monotonic rescaling, since the values are converted to ranks. This also makes it less sensitive to outliers than Pearson.

Of course, these strengths become weaknesses in domains where the raw differences do matter. That said, in most NLU contexts, Spearman will be a good conservative choice for system assessment.

**Related to Spearman rank correlation**

For comparing gold values $y$ and predicted values $\widehat{y}$, Pearson correlation is equivalent to a linear regression using $\textbf{rank}(\widehat{y})$ and a bias term to predict $\textbf{rank}(y)$. [See this great blog post for details.](https://lindeloev.github.io/tests-as-linear/)

### Model Prediction

Now we have defined the performance metrics, we can use them to evaluate the model predctions.

From Train/Test Split Section we have :

- `train_data`, `test_data`, `train_labels` and `test_labels` we can use them for models performance evaluations.

From Model Training section we have : 

- trained instances of `Linear Regressor`, `KNN Regressor`, `Random Forest Regressor` and `Gradient Booster Regressor` in `model_instances` dictionary. We need to feed the test data to these instances  to get the predictions

In [33]:
"""
    get predictions from test instances

    Args:
        trained_model () : trained instance
        test_data (pandas dataframe) : test data without label

    Returns:
        list of predictions against each row in test data
"""


def make_predictions(trained_model, test_data):

    return trained_model.predict(test_data)

    

In [34]:
#store predictions against each model

y_pred_per_model = {}

for model_name, model_instance in model_instances.items():
    
    y_pred_per_model[model_name] = make_predictions(model_instance, test_data) 


We have stored model name and it's predictions over test data in `y_pred_per_model` and we have also `test_labels` which is like ground truths.

### Model Evaluation

This like the last step we need to perform to compare which classification algorithm is performating relatively better over test distribution. In the above sctions, We are done with implementaion of performance metrics now we can use them to evaluate the trained models.

Let's create a report in form of a data frame where headers are `model_name` and the performance metrics we discussed above.

In [41]:
#test labels
y_true = np.array(test_labels)

#lets create a dict which contains model_name and the key pair of metric name and its value
result_dict = {}

#iterate over y_pred_per_model to get model name and corresponding predicted label
for model_name, y_pred in y_pred_per_model.items():

    #find mse
    mse = mean_squared_error(y_true, y_pred)
    
    #find rmse
    rmse = root_mean_squared_error(y_true, y_pred)

    #find mae
    mae = mean_absolute_error(y_true, y_pred)

    #find r2_score
    r2_value = r2_score(y_true, y_pred)

    #find adjusted r2_score
    adjusted_r2_value = adjusted_r2_score(y_true, y_pred, 31)

    result_dict[model_name] = {"mean_squared_error":mse, "root_mean_squared_error":rmse, "mean_absolute_error":mae, "r2_score":r2_value, "adjusted_r2_score":adjusted_r2_value}


#convert result_dict into pandas data frame
result_df = pd.DataFrame.from_dict(result_dict, orient="index")

#disply the result df
result_df.head(20)



,mean_squared_error,root_mean_squared_error,mean_absolute_error,r2_score,adjusted_r2_score
linear_regressor,10.948576,3.308863,2.515675,0.832344,0.476358
random_forest_regressor,13.647620,3.694269,2.595294,0.791013,0.361992
knn_regressor,23.958647,4.894757,3.780789,0.633120,-0.021293
gradient_boosting_regressor,34.745582,5.894538,4.333064,0.467939,-0.331306


##### Observations

From above table, we can deduce that `linear regression` is performing best among all the models across all the metrics. 2nd best model is `random forest regressor`.

Worst performance is given by `Gradient Boost Regressor` across all the metrics.


Note that here I am using the default settings of each model and not doing any hyper parameters tuning. So the above observation is done on default setup of each model and hence can change if we do hyper parameter tunings or optimizations.